In [ ]:
# Normal Comment
# ! Very Important Errors or things which must be fixed or be in attention immediately
# TODO: Things remaining to do
# ? Questions
# * Some Messages or Notes
# ^ Important Notes, Messages or things which may need attention in future
# & Important Commented out Code
# ~ Purple

In [ ]:

#^###############################  Storage_Optimization.ipynb  ####################################
# ^ Author: Sukhendu Sain
# ^ Description: Main file of codebase. Houses main code
# ^ Data: 23-Nov-2024
#^################################################################################

In [ ]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
# df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
# df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
# if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~12-15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service['Description'] = df_Service['Description'].astype(str)
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR')].reset_index()
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Service['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Service['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Dollars Sold'] = df_Service['Qty Sold'].apply(lambda x: [i for i in str(x).strip().split("      ")][-1])
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Qty Sold'] = df_Service['Qty Sold'].apply(lambda x: "     ".join([i for i in str(x).strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [ ]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Processing & Calculation

### Make a Big Final Dataframe

In [ ]:

# * It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from 3 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = [] # Initialize the New List, which will hold all rows before turning into DF
gParts_PartNos = set(df_Gparts['Svc Part Number']) # Get a Set of all Part Nos. of GParts


# Wholesale
common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Wholesale", ac, df_Wholesale[df_Wholesale['Part Number'] == pn]['Sold'].values[0], 0, 0, False, dp, wd, ht, "", "", "", 0, "", ""])


# Service
common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Service", ac, 0, df_Service[df_Service['* indicates a superseded part\nPart Number'] == pn]['Qty Sold'].values[0], 0, False, dp, wd, ht, "", "", "", 0, "", ""])


# Create the Main Dataframe
df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'Part Category', 'DataSource', 'Active', 'Wholesale Sold', 'Service Sold', "Total Sold", '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'OH Inventory', 'Num. Storage Required', "Bin Location"]

# Merge the Parts present in both Service and Wholesale (Duplicates)
gbParts = df_Main.groupby('Part#').count()[df_Main.groupby('Part#').count()['Part Desc.'] == 2].index.to_list()
for pn in gbParts:
    df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Wholesale"), "Service Sold"] = df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Service"), "Service Sold"].values[0]
    df_Main = df_Main[(df_Main['Part#'] != pn) | (df_Main['DataSource'] == "Wholesale")]
# Set 0Dimensions
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
# Drop 0Dimensions Rows if drop0Dims
if drop0Dims: df_Main = df_Main[df_Main["0Dimensions"] == False]
# Set Total_Sold
df_Main["Total Sold"] = df_Main["Wholesale Sold"].astype(int) + df_Main["Service Sold"].astype(int)
# Sort by 'Total Sold'
df_Main = df_Main.sort_values('Total Sold', ascending=False)
# ^ Add Random Values for OH Inventory temporarily
df_Main["OH Inventory"] = np.random.choice(np.arange(20), size=len(df_Main), replace=True)
# Reset Index
df_Main = df_Main.reset_index(drop=True)

In [ ]:

# * Change some data Manually
df_Main.loc[df_Main["Part#"] == '6F2Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["6F2Z1A189A-TIRE",False, 28,28,3,100]
df_Main.loc[df_Main["Part#"] == '7L1Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["7L1Z1A189A-TIRE",False, 34,34,3.5,50]
df_Main.loc[df_Main["Part#"] == '9OO1183106436', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 30,30,3,1000]
df_Main.loc[df_Main["Part#"] == '9OO439510', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 45,45,4.5,500]
df_Main.loc[df_Main["Part#"] == '9OO1732002500', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 50,50,5,250]
df_Main.loc[df_Main["Part#"] == '9OO3004901', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 40,40,4,25]
df_Main[['TIRE' in s for s in df_Main["Part Desc."]]]

In [ ]:
utils.print_df(df_Main)

In [ ]:
def part_categorization(df_toBeCategorized):
    # TODO: Add more categories
    for i in range(df_toBeCategorized.shape[0]):
        desc = "-".join(df_toBeCategorized.loc[i, "Part Desc."].split("-")[1:])
        category = ""
        if "rivet" in desc.lower():
            category = "Rivet"
        elif ("blade" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Blade"        
        elif ("arm" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Arm"
        elif ("v-belt" in desc.lower()):
            category = "V-Belt"
        elif ("belt" in desc.lower()) & ("retractor" not in desc.lower()) & ("hole" not in desc.lower()) & ("cover" not in desc.lower()):
            category = "Belt"
        # elif ("hose" in desc.lower()) & ("vent" not in desc.lower()) & ("connect" not in desc.lower() & ("radiator" not in desc.lower()& ("heater" not in desc.lower()):
        #     category = "Hose"

        df_toBeCategorized.loc[i, 'Part Category'] = category

### Apply Zoning

In [ ]:

## * Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[0]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 0, zoneColName] = None

In [ ]:
## Run the Apply_Zoning on df_Main
Apply_Zoning(df_Main, zones, 'Total Sold', 'Zone')

In [ ]:
## Check each Zone's number of Part Numbers
df_Main['Zone'].value_counts()

### Specialty Storage Assignment

#### Function for Bin Calculation

In [ ]:
def getNumOfBin(depth, width, height, raw_bin_dim, ohInven):
    # ^ Raw Bin Dimensions has this format :-  Height_Depth_Width
    if (raw_bin_dim != "") and (ohInven > 0):
        bin_height = float(raw_bin_dim.split("_")[0])
        bin_depth = float(raw_bin_dim.split("_")[1])
        bin_width = float(raw_bin_dim.split("_")[2])

        numOfBins = math.ceil(ohInven / (math.floor(bin_depth / depth) * math.floor(bin_width / width) * math.floor(bin_height / height)))
        
        return numOfBins
    else:
        return 0

### Main Function for Storage Assignment

In [ ]:
def getStorage(zone, pdesc, depth, width, height, ohInven):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    isSpec, storageType, subStorage, raw_bin_dim = utils.getSpecialityStorage(pdesc, depth, width, height)

    if not isSpec: 
        if (zone == "Red Hot") | (zone == "Red"):
            storageType, subStorage, raw_bin_dim = utils.getRedHotStorage(depth, width, height)
        elif zone == "Orange":
            storageType, subStorage, raw_bin_dim = utils.getOrangeStorage(depth, width, height)
        elif zone == "Yellow":
            storageType, subStorage, raw_bin_dim = utils.getYellowStorage(depth, width, height)
        elif zone == "Green":   
            storageType, subStorage, raw_bin_dim = utils.getGreenStorage(depth, width, height)
        elif zone == "Blue":
            storageType, subStorage, raw_bin_dim = utils.getBlueStorage(depth, width, height)

    numOfBins = getNumOfBin(depth, width, height, raw_bin_dim, ohInven)

    return storageType, subStorage, numOfBins # Return the Values

#### Apply the Storage Function

In [ ]:
for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pdesc = df_Main.loc[i, "Part Desc."]
    ohInven = df_Main.loc[i, "OH Inventory"]

    # If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"], df_Main.loc[i, "Num. Storage Required"] = getStorage(zone, pdesc, depth, width, height, ohInven)


In [ ]:
import math

def optimize_part_packing(part_height, part_width, part_depth, shelf_height, shelf_width, shelf_depth):
    combinations = [
        ("width", "height", "depth"),  # Part_width along shelf_width, part_height along shelf_height, and part_depth along shelf_depth
        ("width", "depth", "height"),  # Part_width along shelf_depth, part_height along shelf_height, and part_depth along shelf_width
        ("depth", "width", "height"),  # Part_depth along shelf_width, part_height along shelf_height, and part_width along shelf_depth
        ("depth", "height", "width"),  # Part_depth along shelf_depth, part_height along shelf_height, and part_width along shelf_width
        ("height", "depth", "width"),  # Part_width along shelf_height, part_height along shelf_depth, and part_depth along shelf_width
        ("height", "width", "depth"),  # Part_depth along shelf_height, part_height along shelf_depth, and part_width along shelf_width
     ]

    results = []

    def get_dimension(dimensions, index):
        return {
            'width': dimensions[1],
            'height': dimensions[0],
            'depth': dimensions[2]
        }[index]

    part_dimensions = (part_height, part_width, part_depth)
    shelf_dimensions = (shelf_height, shelf_width, shelf_depth)

    for combination in combinations:
        p1, p2, p3 = [get_dimension(part_dimensions, c) for c in combination]
        s1 = shelf_width
        s2 = shelf_depth
        s3 = shelf_height
        
        initial_layers = math.floor(s1 / p1)
        initial_filters_per_layer = math.floor(s2 / p2) * math.floor(s3 / p3)
        initial_parts = initial_layers * initial_filters_per_layer

        remaining_s1 = s1 % p1
        remaining_s2 = s2 % p2
        remaining_s3 = s3 % p3

        additional_parts = calculate_additional_parts(remaining_s1, remaining_s2, remaining_s3, part_height, part_width, part_depth)

        total_parts = initial_parts + additional_parts

        results.append({
            'Combination': f"Part_{combination[0]} along shelf_width, Part_{combination[1]} along shelf_depth, and Part_{combination[2]} along shelf_height",
            'Initial Parts': initial_parts,
            'Additional Parts': additional_parts,
            'Total Parts': total_parts
        })

    return results

def calculate_additional_parts(remaining_height, remaining_width, remaining_depth, part_height, part_width, part_depth):
    max_filters = 0
    
    orientations = [
        (part_height, part_width, part_depth),
        (part_height, part_depth, part_width),
        (part_depth, part_width, part_height),
        (part_depth, part_height, part_width),
        (part_width, part_height, part_depth),
        (part_width, part_depth, part_height)
    ]
    
    for orientation in orientations:
        new_layers = math.floor(remaining_height / orientation[0])
        new_filters_per_layer = math.floor(remaining_width / orientation[1]) * math.floor(remaining_depth / orientation[2])
        
        additional_filters = new_layers * new_filters_per_layer
        max_filters = max(max_filters, additional_filters)
    
    return max_filters

# Example usage
part_height = 2
part_width = 8
part_depth = 10

shelf_height = 12
shelf_width = 12
shelf_depth = 48

results = optimize_part_packing(part_height, part_width, part_depth, shelf_height, shelf_width, shelf_depth)

for result in results:
    print(f"\n{result['Combination']}:")
    print(f"Initial Parts: {result['Initial Parts']}")
    print(f"Additional Parts: {result['Additional Parts']}")
    print(f"Total Parts: {result['Total Parts']}")

print("\nBest Combination:")
best_result = max(results, key=lambda x: x['Total Parts'])
print(f"{best_result['Combination']}: Total Parts = {best_result['Total Parts']}")

In [ ]:
def calculate_max_parts(shelf_depth, shelf_width, shelf_height, part_depth, part_width, part_height):
    print(f"Initial call: shelf_depth={shelf_depth}, shelf_width={shelf_width}, shelf_height={shelf_height}, "
          f"part_depth={part_depth}, part_width={part_width}, part_height={part_height}")

    if (part_depth * part_width * part_height) > (shelf_depth * shelf_width * shelf_height):
        print("Returning 0 due to invalid dimensions")
        return 0, None

    def calculate_parts_for_orientation(orientation):
        print(f"Calculating for orientation: {orientation}")
        if orientation == 'depth':
            if (part_depth > shelf_depth) or (part_width > shelf_width) or (part_height > shelf_height): return 0
            base_parts = ((shelf_depth // part_depth) * (shelf_width // part_width)) * (shelf_height // part_height)
            print(f"Base parts for depth: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_depth, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_width, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_height, part_depth, part_width, part_height)
            print(f"Additional parts for depth: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)
        elif orientation == 'height':
            if (part_height > shelf_depth) or (part_width > shelf_width) or (part_depth > shelf_height): return 0
            base_parts = ((shelf_depth // part_height) * (shelf_width // part_width)) * (shelf_height // part_depth)
            print(f"Base parts for height: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_height, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_width, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_depth, part_depth, part_width, part_height)
            print(f"Additional parts for height: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)
        elif orientation == 'width':
            if (part_width > shelf_depth) or (part_depth > shelf_width) or (part_height > shelf_height): return 0
            base_parts = ((shelf_depth // part_width) * (shelf_width // part_depth)) * (shelf_height // part_height)
            print(f"Base parts for width: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_width, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_depth, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_height, part_depth, part_width, part_height)
            print(f"Additional parts for width: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)
        elif orientation == 'depth_height':
            if (part_depth > shelf_depth) or (part_height > shelf_width) or (part_width > shelf_height): return 0
            base_parts = ((shelf_depth // part_depth) * (shelf_width // part_height)) * (shelf_height // part_width)
            print(f"Base parts for depth_height: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_depth, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_height, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_width, part_depth, part_width, part_height)
            print(f"Additional parts for depth_height: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)
        elif orientation == 'depth_width':
            if (part_height > shelf_depth) or (part_depth > shelf_width) or (part_width > shelf_height): return 0
            base_parts = ((shelf_depth // part_height) * (shelf_width // part_depth)) * (shelf_height // part_width)
            print(f"Base parts for depth_width: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_height, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_height, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_width, part_depth, part_width, part_height)
            print(f"Additional parts for depth_width: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)
        elif orientation == 'height_width':
            if (part_width > shelf_depth) or (part_height > shelf_width) or (part_depth > shelf_height): return 0
            base_parts = ((shelf_depth // part_width) * (shelf_width // part_height)) * (shelf_height // part_depth)
            print(f"Base parts for height_width: {base_parts}")
            additional_parts1, best_orient = calculate_max_parts(shelf_depth % part_width, shelf_width, shelf_height, part_depth, part_width, part_height)
            additional_parts2, best_orient = calculate_max_parts(shelf_depth, shelf_width % part_height, shelf_height, part_depth, part_width, part_height)
            additional_parts3, best_orient = calculate_max_parts(shelf_depth, shelf_width, shelf_height % part_depth, part_depth, part_width, part_height)
            print(f"Additional parts for height_width: {additional_parts1}, {additional_parts2}, {additional_parts3}")
            return base_parts + max(additional_parts1, additional_parts2, additional_parts3)

    orientations = ['depth', 'height', 'width', 'depth_height', 'depth_width', 'height_width']
    
    max_parts = 0
    best_orientation = None
    
    for orientation in orientations:
        parts = calculate_parts_for_orientation(orientation)
        print(f"Total parts for {orientation}: {parts}")
        if parts > max_parts:
            max_parts = parts
            best_orientation = orientation
    
    print(f"Final result: Max parts = {max_parts}, Best orientation = {best_orientation}")
    return max_parts, best_orientation

# Example usage
shelf_depth = 12
shelf_width = 48
shelf_height = 12
part_depth = 8
part_width = 8
part_height = 2

max_parts, best_orientation = calculate_max_parts(shelf_depth, shelf_width, shelf_height, part_depth, part_width, part_height)

print(f"\nMaximum number of parts: {max_parts}")
print(f"Best orientation: {best_orientation}")

In [ ]:
def calculate_max_parts(shelf_depth, shelf_width, shelf_height, part_depth, part_width, part_height):
    def calculate_base_parts(orientation):
        if orientation == 'depth':
            return ((shelf_depth // part_depth) * (shelf_width // part_width)) * (shelf_height // part_height)
        elif orientation == 'height':
            return ((shelf_depth // part_height) * (shelf_width // part_width)) * (shelf_height // part_depth)
        elif orientation == 'width':
            return ((shelf_depth // part_width) * (shelf_width // part_depth)) * (shelf_height // part_height)
        elif orientation == 'depth_height':
            return ((shelf_depth // part_depth) * (shelf_width // part_height)) * (shelf_height // part_width)
        elif orientation == 'depth_width':
            return ((shelf_depth // part_depth) * (shelf_width // part_width)) * (shelf_height // part_height)
        elif orientation == 'height_width':
            return ((shelf_depth // part_height) * (shelf_width // part_width)) * (shelf_height // part_depth)
        elif orientation == 'width_depth':
            return ((shelf_depth // part_width) * (shelf_width // part_depth)) * (shelf_height // part_height)
        elif orientation == 'width_height':
            return ((shelf_depth // part_width) * (shelf_width // part_height)) * (shelf_height // part_depth)
    
        
    orientations = ['depth', 'height', 'width', 'depth_height', 'depth_width', 'height_width']
    
    max_parts = 0
    best_orientation = None
    
    for orientation in orientations:
        base_parts = calculate_base_parts(orientation)
        
        total_parts = base_parts
        
        if total_parts > max_parts:
            max_parts = total_parts
            best_orientation = orientation
    
    return max_parts, best_orientation
    

# Example usage
shelf_depth = 12
shelf_width = 48
shelf_height = 12
part_depth = 8
part_width = 8
part_height = 2

max_parts = calculate_max_parts(shelf_depth, shelf_width, shelf_height, part_depth, part_width, part_height)

print(f"\nMaximum number of parts: {max_parts[0]}")
print(f"\nBest Orientation: {max_parts[1]}")

In [ ]:

## ^ Tire Storage Calculation
# TODO: CLARIFY the Calculation of Tire Carousel Model Selection Based on Percentage 
## * ASSUMPTION:  If 50% of Tires Have 33-inch or More Diameter, Assign Large-Storage (72-width carousel)
## * ELSE,  For 28-inch or less, and,  between 28-33 inches, assign standard carrousel (48-width carousel) 
# & df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', 'SubStorage'] 
carousel_model = '72-inch Wide' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '>33-inches Wide'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= TIRE_PERCENT1 else '48-inch Wide'
carousel_width = '72' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '>33-inches Wide'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= TIRE_PERCENT1 else '48'
for tirePN in df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', "Part#"]:
    df_Main.loc[(df_Main['Part#'] == tirePN), "Num. Storage Required"] = getNumOfBin(df_Main.loc[(df_Main['Part#'] == tirePN), "Depth"].values[0], df_Main.loc[(df_Main['Part#'] == tirePN), "Width"].values[0], df_Main.loc[(df_Main['Part#'] == tirePN), "Height"].values[0], f"{df_Main.loc[(df_Main['Part#'] == tirePN), "Height"].values[0]}_{df_Main.loc[(df_Main['Part#'] == tirePN), "Depth"].values[0]}_{carousel_width}", df_Main.loc[(df_Main['Part#'] == tirePN), "OH Inventory"].values[0])

In [ ]:
utils.print_df(df_Main)

In [ ]:
df_Main.to_excel('FinalDataset.xlsx', index=False) 

In [ ]:
df_Main["StorageType"].value_counts()

In [ ]:
df_Main[(df_Main["0Dimensions"] == True) & (df_Main["StorageType"] == "")]